In [133]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import ttest_ind, norm, chi2_contingency, f_oneway

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

from unidecode import unidecode

## ETL

En este caso trabajas en una empresa de venta al por menor de productos italianos y debes realizar la limpieza, transformación e integración de datos de ventas, productos y clientes para su análisis.

1. Lectura de la Información:

    - Leer los archivos CSV (ventas.csv, productos.csv, clientes.csv).

    - Explorar los conjuntos de datos para comprender su estructura, columnas, tipos de datos, etc.

2. Transformación de Datos:

    - Limpiar los datos: manejar valores nulos, eliminar duplicados si los hay, corregir errores tipográficos, etc.

    - Realizar la integración de datos: unir los conjuntos de datos apropiados para obtener una tabla única que contenga información de ventas junto con detalles de productos y clientes.

    - Aplicar transformaciones relevantes según sea necesario: por ejemplo, convertir tipos de datos, renombrar columnas, crear nuevas características derivadas, etc.

In [104]:
with open("files/productos.csv", 'r') as file:

    lines = file.readlines()

In [105]:
def unificar(lines):
    lista_columnas = lines[0][0:-1].split(",")
    #print(lista_columnas)
    lista_productos = []
    for line in lines[1:-1]:
        split = line.split(",")
        join = ",".join(split[5:-1])
        all = split[0:5]
        all.append(join)
        lista_productos.append(all)
    df_all = pd.DataFrame(lista_productos)
    df_all.columns = lista_columnas
    df_all = df_all.set_index("ID")
    return df_all.to_csv("files/productos1.csv")
    

In [106]:
unificar(lines)

In [107]:
split = lines[8].split(",")
join = ",".join(split[5:])
all = split[0:5]
all.append(join)
print(all)


['H8', 'Pesto Genovese', 'Salsas', '5.99', 'Italia', 'Una salsa italiana de albahaca, piñones, queso parmesano y aceite de oliva, perfecta para pastas.\n']


In [108]:
def apertura_fichero(ruta):
    csv_apertura= pd.read_csv(ruta, index_col=0)
    csv_apertura.reset_index(inplace = True)
    display(csv_apertura.head())
    return csv_apertura


In [109]:
# se puede revisar y mejorar
def exploracion_datos(csv):
    forma = csv.shape
    print(f"La forma es {forma}")
    print("_______________")
    nulos = csv.isna().sum().reset_index()
    print(f"Los nulos son:")
    display(nulos)
    print("_______________")
    duplicados = csv.duplicated().sum()
    print(f"Hay {duplicados} duplicados")
    print("_______________")
    tipo_dato = csv.dtypes.reset_index()
    print(f"Los datos son de tipo:")
    display(tipo_dato)
    print("_______________")
    columnas = csv.columns
    print(f"Las columnas son {columnas}")

In [110]:
#para customers tenemos que hacer gestion nulos en columnas categoricas

In [111]:
customers = apertura_fichero("files/clientes.csv")

,id,first_name,last_name,email,gender,City,Country,Address
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
2,3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing
3,4,Filmer,Eirwin,feirwin3@intel.com,NaN,Leon,Spain,5 American Ash Road
4,5,Tanhya,Lubbock,tlubbock4@huffingtonpost.com,Female,"Hospitalet De Llobregat, L'",Spain,9289 Merry Circle


In [112]:
exploracion_datos(customers)

La forma es (1000, 8)
_______________
Los nulos son:


,index,0
0,id,0
1,first_name,0
2,last_name,0
3,email,27
4,gender,77
5,City,124
6,Country,154
7,Address,41


_______________
Hay 0 duplicados
_______________
Los datos son de tipo:


,index,0
0,id,int64
1,first_name,object
2,last_name,object
3,email,object
4,gender,object
5,City,object
6,Country,object
7,Address,object


_______________
Las columnas son Index(['id', 'first_name', 'last_name', 'email', 'gender', 'City', 'Country',
       'Address'],
      dtype='object')


In [113]:
#def customer_correcion(df, columna):
    #df[columna] = df[columna].fillna("Desconocido")
    #for line in df[columna]:
        #split = line.split(",")
    #return split
        #df[columna] = split[0]
        #print(df[columna])
        #print(customers["City"])
        #lista_ciudades.append(all)
    #return customers["City"]

    
  

In [114]:
#nulos_customer = [columna in customers.columns if columna.isnull]

In [115]:
#def conversion_nulos(df, lista):
    #for columna in lista:
        #df[columna] = df[columna].fillna("Desconocido")

In [116]:
sales = apertura_fichero("files/ventas.csv")

,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total
0,723,A1,2023-11-22,2,17.98
1,498,C3,2023-11-21,1,5.49
2,121,D4,2023-11-20,3,32.97
3,885,L12,2023-11-19,1,6.49
4,347,Q17,2023-11-18,2,7.98


In [117]:
exploracion_datos(sales)

La forma es (100, 5)
_______________
Los nulos son:


,index,0
0,ID_Cliente,0
1,ID_Producto,0
2,Fecha_Venta,0
3,Cantidad,0
4,Total,0


_______________
Hay 0 duplicados
_______________
Los datos son de tipo:


,index,0
0,ID_Cliente,int64
1,ID_Producto,object
2,Fecha_Venta,object
3,Cantidad,int64
4,Total,float64


_______________
Las columnas son Index(['ID_Cliente', 'ID_Producto', 'Fecha_Venta', 'Cantidad', 'Total'], dtype='object')


In [118]:
items = apertura_fichero("files/productos1.csv")

,ID,Nombre_Producto,Categoría,Precio,Origen,Descripción
0,A1,Pizza Margherita,Platos Preparados,8.99,Italia,Clásica pizza italiana con tomate
1,B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,Risotto cremoso con champiñones frescos
2,C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...
3,D4,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...
4,E5,Orecchiette,Productos Secos,4.29,Italia,NaN


In [119]:
exploracion_datos(items)

La forma es (33, 6)
_______________
Los nulos son:


,index,0
0,ID,0
1,Nombre_Producto,0
2,Categoría,0
3,Precio,0
4,Origen,0
5,Descripción,1


_______________
Hay 0 duplicados
_______________
Los datos son de tipo:


,index,0
0,ID,object
1,Nombre_Producto,object
2,Categoría,object
3,Precio,float64
4,Origen,object
5,Descripción,object


_______________
Las columnas son Index(['ID', 'Nombre_Producto', 'Categoría', 'Precio', 'Origen',
       'Descripción'],
      dtype='object')


In [120]:
 #Renombrar columnas

items.rename(columns = {"ID" : "ID_Producto"}, inplace = True)


In [121]:
customers.head()

,id,first_name,last_name,email,gender,City,Country,Address
0,1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
1,2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
2,3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing
3,4,Filmer,Eirwin,feirwin3@intel.com,NaN,Leon,Spain,5 American Ash Road
4,5,Tanhya,Lubbock,tlubbock4@huffingtonpost.com,Female,"Hospitalet De Llobregat, L'",Spain,9289 Merry Circle


In [122]:
customers.rename(columns = {
        'id': 'ID_Cliente',
        'first_name': 'nombre',
        'last_name': 'apellido',
        'email': 'correo',
        'gender': 'género',
        'City': 'Ciudad',
        'Country': 'País',
        'Address': 'Dirección'
    }, inplace = True)

In [123]:
sales.head()

,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total
0,723,A1,2023-11-22,2,17.98
1,498,C3,2023-11-21,1,5.49
2,121,D4,2023-11-20,3,32.97
3,885,L12,2023-11-19,1,6.49
4,347,Q17,2023-11-18,2,7.98


In [124]:
#Unificar ficheros

first_uni= sales.merge(items, on="ID_Producto", how="left")

In [125]:
first_uni.head(5)

,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total,Nombre_Producto,Categoría,Precio,Origen,Descripción
0,723,A1,2023-11-22,2,17.98,Pizza Margherita,Platos Preparados,8.99,Italia,Clásica pizza italiana con tomate
1,498,C3,2023-11-21,1,5.49,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...
2,121,D4,2023-11-20,3,32.97,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...
3,885,L12,2023-11-19,1,6.49,Sardinas en Aceite de Oliva,Pescados en Conserva,3.99,Italia,Sardinas frescas en aceite de oliva
4,347,Q17,2023-11-18,2,7.98,Pasta de Trigo,Productos Secos,3.99,Italia,Deliciosa pasta de trigo durum italiana


In [126]:
data_final = merged_df.merge(customers, on="ID_Cliente")

In [127]:
data_final

,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total,Nombre_Producto,Categoría,Precio,Origen,Descripción,nombre,apellido,correo,género,Ciudad,País,Dirección
0,723,A1,2023-11-22,2,17.98,Pizza Margherita,Platos Preparados,8.99,Italia,Clásica pizza italiana con tomate,Sephira,Bleackley,sbleackleyk2@liveinternet.ru,NaN,Girona,NaN,4 Birchwood Road
1,498,C3,2023-11-21,1,5.49,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...,Padgett,Painter,ppainterdt@scientificamerican.com,Male,Barcelona,Spain,5 Artisan Lane
2,121,D4,2023-11-20,3,32.97,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...,Wye,Sinncock,wsinncock3c@fc2.com,Male,Santander,NaN,4679 Fair Oaks Trail
3,885,L12,2023-11-19,1,6.49,Sardinas en Aceite de Oliva,Pescados en Conserva,3.99,Italia,Sardinas frescas en aceite de oliva,Windham,Badham,wbadhamok@wired.com,Male,"Hospitalet De Llobregat, L'",Spain,68 Nelson Junction
4,347,Q17,2023-11-18,2,7.98,Pasta de Trigo,Productos Secos,3.99,Italia,Deliciosa pasta de trigo durum italiana,Egbert,Leborgne,eleborgne9m@reuters.com,Male,Sevilla,Spain,7 Quincy Way
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,798,AX50,2023-10-21,2,13.98,NaN,NaN,NaN,NaN,NaN,Chandler,MacAirt,cmacairtm5@google.co.uk,Male,Valladolid,Spain,39103 Buell Circle
96,798,BI77,2023-11-02,2,14.50,NaN,NaN,NaN,NaN,NaN,Chandler,MacAirt,cmacairtm5@google.co.uk,Male,Valladolid,Spain,39103 Buell Circle
97,465,AY51,2023-10-20,1,7.25,NaN,NaN,NaN,NaN,NaN,Gerrie,MacGauhy,gmacgauhycw@storify.com,Genderqueer,Valencia,Spain,4 Tennyson Parkway
98,456,BM81,2023-10-29,3,26.97,NaN,NaN,NaN,NaN,NaN,Sissy,Armatidge,sarmatidgecn@studiopress.com,NaN,Jaen,Spain,85877 Del Sol Way


In [128]:
exploracion_datos(data_final)

La forma es (100, 17)
_______________
Los nulos son:


,index,0
0,ID_Cliente,0
1,ID_Producto,0
2,Fecha_Venta,0
3,Cantidad,0
4,Total,0
5,Nombre_Producto,90
6,Categoría,90
7,Precio,90
8,Origen,90
9,Descripción,90


_______________
Hay 0 duplicados
_______________
Los datos son de tipo:


,index,0
0,ID_Cliente,int64
1,ID_Producto,object
2,Fecha_Venta,object
3,Cantidad,int64
4,Total,float64
5,Nombre_Producto,object
6,Categoría,object
7,Precio,float64
8,Origen,object
9,Descripción,object


_______________
Las columnas son Index(['ID_Cliente', 'ID_Producto', 'Fecha_Venta', 'Cantidad', 'Total',
       'Nombre_Producto', 'Categoría', 'Precio', 'Origen', 'Descripción',
       'nombre', 'apellido', 'correo', 'género', 'Ciudad', 'País',
       'Dirección'],
      dtype='object')


In [129]:
data_final.columns = data_final.columns.str.lower()


In [130]:
data_final

,id_cliente,id_producto,fecha_venta,cantidad,total,nombre_producto,categoría,precio,origen,descripción,nombre,apellido,correo,género,ciudad,país,dirección
0,723,A1,2023-11-22,2,17.98,Pizza Margherita,Platos Preparados,8.99,Italia,Clásica pizza italiana con tomate,Sephira,Bleackley,sbleackleyk2@liveinternet.ru,NaN,Girona,NaN,4 Birchwood Road
1,498,C3,2023-11-21,1,5.49,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...,Padgett,Painter,ppainterdt@scientificamerican.com,Male,Barcelona,Spain,5 Artisan Lane
2,121,D4,2023-11-20,3,32.97,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...,Wye,Sinncock,wsinncock3c@fc2.com,Male,Santander,NaN,4679 Fair Oaks Trail
3,885,L12,2023-11-19,1,6.49,Sardinas en Aceite de Oliva,Pescados en Conserva,3.99,Italia,Sardinas frescas en aceite de oliva,Windham,Badham,wbadhamok@wired.com,Male,"Hospitalet De Llobregat, L'",Spain,68 Nelson Junction
4,347,Q17,2023-11-18,2,7.98,Pasta de Trigo,Productos Secos,3.99,Italia,Deliciosa pasta de trigo durum italiana,Egbert,Leborgne,eleborgne9m@reuters.com,Male,Sevilla,Spain,7 Quincy Way
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,798,AX50,2023-10-21,2,13.98,NaN,NaN,NaN,NaN,NaN,Chandler,MacAirt,cmacairtm5@google.co.uk,Male,Valladolid,Spain,39103 Buell Circle
96,798,BI77,2023-11-02,2,14.50,NaN,NaN,NaN,NaN,NaN,Chandler,MacAirt,cmacairtm5@google.co.uk,Male,Valladolid,Spain,39103 Buell Circle
97,465,AY51,2023-10-20,1,7.25,NaN,NaN,NaN,NaN,NaN,Gerrie,MacGauhy,gmacgauhycw@storify.com,Genderqueer,Valencia,Spain,4 Tennyson Parkway
98,456,BM81,2023-10-29,3,26.97,NaN,NaN,NaN,NaN,NaN,Sissy,Armatidge,sarmatidgecn@studiopress.com,NaN,Jaen,Spain,85877 Del Sol Way


In [134]:
data_final.columns = [unidecode(columna) for columna in data_final.columns]

In [135]:
data_final

,id_cliente,id_producto,fecha_venta,cantidad,total,nombre_producto,categoria,precio,origen,descripcion,nombre,apellido,correo,genero,ciudad,pais,direccion
0,723,A1,2023-11-22,2,17.98,Pizza Margherita,Platos Preparados,8.99,Italia,Clásica pizza italiana con tomate,Sephira,Bleackley,sbleackleyk2@liveinternet.ru,NaN,Girona,NaN,4 Birchwood Road
1,498,C3,2023-11-21,1,5.49,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...,Padgett,Painter,ppainterdt@scientificamerican.com,Male,Barcelona,Spain,5 Artisan Lane
2,121,D4,2023-11-20,3,32.97,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...,Wye,Sinncock,wsinncock3c@fc2.com,Male,Santander,NaN,4679 Fair Oaks Trail
3,885,L12,2023-11-19,1,6.49,Sardinas en Aceite de Oliva,Pescados en Conserva,3.99,Italia,Sardinas frescas en aceite de oliva,Windham,Badham,wbadhamok@wired.com,Male,"Hospitalet De Llobregat, L'",Spain,68 Nelson Junction
4,347,Q17,2023-11-18,2,7.98,Pasta de Trigo,Productos Secos,3.99,Italia,Deliciosa pasta de trigo durum italiana,Egbert,Leborgne,eleborgne9m@reuters.com,Male,Sevilla,Spain,7 Quincy Way
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,798,AX50,2023-10-21,2,13.98,NaN,NaN,NaN,NaN,NaN,Chandler,MacAirt,cmacairtm5@google.co.uk,Male,Valladolid,Spain,39103 Buell Circle
96,798,BI77,2023-11-02,2,14.50,NaN,NaN,NaN,NaN,NaN,Chandler,MacAirt,cmacairtm5@google.co.uk,Male,Valladolid,Spain,39103 Buell Circle
97,465,AY51,2023-10-20,1,7.25,NaN,NaN,NaN,NaN,NaN,Gerrie,MacGauhy,gmacgauhycw@storify.com,Genderqueer,Valencia,Spain,4 Tennyson Parkway
98,456,BM81,2023-10-29,3,26.97,NaN,NaN,NaN,NaN,NaN,Sissy,Armatidge,sarmatidgecn@studiopress.com,NaN,Jaen,Spain,85877 Del Sol Way
